In [5]:
from src.ic_device_llm.utils.json_utils import load_jsonl_files
from openai import OpenAI
from src.ic_device_llm.configs.api_key_config import OPENAI_API_KEY
from src.ic_device_llm.configs.path_config import RESOURCES_DIR_PATH
from loguru import logger

In [6]:
train_data_file_path = RESOURCES_DIR_PATH / "openai_conversations.jsonl"
all_data_files_path = RESOURCES_DIR_PATH / "openai_conversations_all.jsonl"
train_data = load_jsonl_files(train_data_file_path)
all_data = load_jsonl_files(all_data_files_path)
logger.info(f"Train data length: {len(train_data)}")
logger.info(f"All data length: {len(all_data)}")

2024-07-17 12:49:24.907 | INFO     | __main__:<module>:5 - Train data length: 385
2024-07-17 12:49:24.908 | INFO     | __main__:<module>:6 - All data length: 86625


In [23]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [27]:
# def retrieve_ids_from_str(input_str: str) -> float:
# "    The ids of this device is 2.1636e-05 A"

import re
def retrieve_ids_from_str(input_str: str) -> float:
    # Use a regular expression to find the floating-point number in scientific notation
    match = re.search(r'(\d+\.\d+e[-+]?\d+)', input_str)
    
    if match:
        # Convert the matched string to a float
        return float(match.group(1))
    else:
        # Return None or raise an exception if no match is found
        return None  # or raise ValueError("No valid ID found in the string")
def extract_gaa_parameters(input_str: str) -> dict:
    # Define patterns for each parameter
    patterns = {
        'lg_nw': r'Gate length \(lg_nw\):\s*([\d.]+)\s*nm',
        'r_nw': r'Nanowire radius \(r_nw\):\s*([\d.]+)\s*nm',
        't_ox_nm': r'Oxide thickness \(t_ox_nm\):\s*([\d.]+)\s*nm'
    }
    
    # Initialize a dictionary to store the extracted values
    params = {}
    
    # Extract each parameter
    for key, pattern in patterns.items():
        match = re.search(pattern, input_str)
        if match:
            params[key] = float(match.group(1))
        else:
            params[key] = None  # or you could raise an exception
    
    return params
retrieve_ids_from_str("    The ids of this device is 2.1636e-05 A")

2.1636e-05

In [29]:
input_string = """  {"messages": [{"role": "system", "content": "You are a helpful assistant on GAA devices."}, {"role": "user", "content": "    GAA device parameters:\n        - Gate length (lg_nw): 10.0 nm\n        - Nanowire radius (r_nw): 4.0 nm\n        - Oxide thickness (t_ox_nm): 0.75 nm\n        - Gate voltage (vg): 0.25 V\n        - Drain voltage (vd): 0.1 V\n    Infer the drain saturation current (ids) for this GAA device."}, {"role": "assistant", "content": "    The ids of this device is 1.9101e-06 A\n    "}]}
"""

parameters = extract_gaa_parameters(input_string)
print(parameters)


{'lg_nw': 10.0, 'r_nw': 4.0, 't_ox_nm': 0.75}


In [18]:
train_data_all_user_input = [item["messages"][1]["content"].strip() for item in train_data]
all_data_all_user_input = [item["messages"][1]["content"].strip()  for item in all_data]

In [34]:
import json

# any way ， 直接测试一下在训练集上的效果了
results = []
from tqdm import tqdm
for item in tqdm(train_data):
    try:
        messages = item["messages"]
        input_messages = messages[:-1]
        assistant_message_str = messages[-1]["content"]
        user_message_str = messages[1]["content"]
        grounding_truth_ids = retrieve_ids_from_str(assistant_message_str)
        parameters = extract_gaa_parameters(user_message_str)
        result = {
                **parameters,
                "ids_grounding_truth": grounding_truth_ids,
                # "predicted": predicted_ids  # Uncomment this when you have predictions
            }       
        response = client.chat.completions.create(
              model="ft:gpt-3.5-turbo-0125:midsummer::9lpms7Qc",
              messages=input_messages 
            )
        predicted_ids_str = response.choices[0].message.content
        predicted_ids = retrieve_ids_from_str(predicted_ids_str)
        result["ids_predicted"] = predicted_ids
        results.append(result)
        logger.info(f"Result: {result}")
    except Exception as e:
        logger.error(f"Error: {e}")
        continue


100%|██████████| 385/385 [05:50<00:00,  1.10it/s]


In [37]:
res_string = """  0%|          | 0/385 [00:00<?, ?it/s][32m2024-07-17 13:44:31.236[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 2.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.0439e-09, 'ids_predicted': 3.5847e-07}[0m
  0%|          | 1/385 [00:01<06:44,  1.05s/it][32m2024-07-17 13:44:32.407[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 2.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 5.4966e-06, 'ids_predicted': 1.2206e-05}[0m
  1%|          | 2/385 [00:02<07:09,  1.12s/it][32m2024-07-17 13:44:33.473[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 2.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.797e-05, 'ids_predicted': 2.2974e-05}[0m
  1%|          | 3/385 [00:03<06:58,  1.10s/it][32m2024-07-17 13:44:34.385[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 2.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.3057e-05, 'ids_predicted': 1.2709e-05}[0m
  1%|          | 4/385 [00:04<06:29,  1.02s/it][32m2024-07-17 13:44:37.734[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 2.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.5352e-09, 'ids_predicted': 1.563e-10}[0m
  1%|▏         | 5/385 [00:07<11:47,  1.86s/it][32m2024-07-17 13:44:38.868[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 2.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 3.2673e-06, 'ids_predicted': 2.7331e-09}[0m
  2%|▏         | 6/385 [00:08<10:11,  1.61s/it][32m2024-07-17 13:44:40.206[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 2.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 7.1421e-06, 'ids_predicted': 8.5655e-06}[0m
  2%|▏         | 7/385 [00:10<09:36,  1.52s/it][32m2024-07-17 13:44:41.078[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 2.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 2.1557e-05, 'ids_predicted': 2.2304e-05}[0m
  2%|▏         | 8/385 [00:10<08:16,  1.32s/it][32m2024-07-17 13:44:43.806[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 2.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 6.3412e-09, 'ids_predicted': 3.1872e-07}[0m
  2%|▏         | 9/385 [00:13<11:01,  1.76s/it][32m2024-07-17 13:44:45.886[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 2.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 9.9002e-09, 'ids_predicted': 7.0109e-09}[0m
  3%|▎         | 10/385 [00:15<11:36,  1.86s/it][32m2024-07-17 13:44:46.637[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 3.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 4.8034e-06, 'ids_predicted': 9.8328e-06}[0m
  3%|▎         | 11/385 [00:16<09:27,  1.52s/it][32m2024-07-17 13:44:48.747[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 3.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 9.7356e-06, 'ids_predicted': 5.0516e-05}[0m
  3%|▎         | 12/385 [00:18<10:33,  1.70s/it][32m2024-07-17 13:44:49.634[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 3.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.346e-05, 'ids_predicted': 1.5337e-05}[0m
  3%|▎         | 13/385 [00:19<09:00,  1.45s/it][32m2024-07-17 13:44:50.364[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 3.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 9.0696e-06, 'ids_predicted': 1.786e-05}[0m
  4%|▎         | 14/385 [00:20<07:37,  1.23s/it][32m2024-07-17 13:44:51.112[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 3.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 4.0349e-06, 'ids_predicted': 7.4223e-06}[0m
  4%|▍         | 15/385 [00:20<06:42,  1.09s/it][32m2024-07-17 13:44:51.860[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 3.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 2.0916e-06, 'ids_predicted': 8.0622e-07}[0m
  4%|▍         | 16/385 [00:21<06:03,  1.01it/s][32m2024-07-17 13:44:52.565[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 3.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.0555e-06, 'ids_predicted': 1.1312e-07}[0m
  4%|▍         | 17/385 [00:22<05:31,  1.11it/s][32m2024-07-17 13:44:53.296[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 3.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.0626e-05, 'ids_predicted': 1.062e-05}[0m
  5%|▍         | 18/385 [00:23<05:11,  1.18it/s][32m2024-07-17 13:44:53.999[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 3.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.1047e-05, 'ids_predicted': 2.5427e-05}[0m
  5%|▍         | 19/385 [00:23<04:55,  1.24it/s][32m2024-07-17 13:44:56.418[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 3.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 7.3715e-12, 'ids_predicted': 5.2368e-10}[0m
  5%|▌         | 20/385 [00:26<07:50,  1.29s/it][32m2024-07-17 13:44:57.223[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 4.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 2.1681e-05, 'ids_predicted': 3.8533e-05}[0m
  5%|▌         | 21/385 [00:27<06:56,  1.14s/it][32m2024-07-17 13:44:58.129[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 4.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.9101e-06, 'ids_predicted': 7.5688e-07}[0m
  6%|▌         | 22/385 [00:27<06:29,  1.07s/it][32m2024-07-17 13:44:59.344[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 4.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 2.4041e-06, 'ids_predicted': 2.9969e-06}[0m
  6%|▌         | 23/385 [00:29<06:43,  1.12s/it][32m2024-07-17 13:44:59.998[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 4.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.732e-05, 'ids_predicted': 2.3672e-05}[0m
  6%|▌         | 24/385 [00:29<05:52,  1.02it/s][32m2024-07-17 13:45:00.684[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 4.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.2337e-05, 'ids_predicted': 3.4483e-05}[0m
  6%|▋         | 25/385 [00:30<05:20,  1.12it/s][32m2024-07-17 13:45:01.416[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 4.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 5.4299e-06, 'ids_predicted': 3.0329e-06}[0m
  7%|▋         | 26/385 [00:31<05:02,  1.19it/s][32m2024-07-17 13:45:02.367[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 4.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 7.6015e-06, 'ids_predicted': 5.0587e-06}[0m
  7%|▋         | 27/385 [00:32<05:13,  1.14it/s][32m2024-07-17 13:45:03.117[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 4.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 6.6964e-07, 'ids_predicted': 1.6782e-07}[0m
  7%|▋         | 28/385 [00:32<04:58,  1.19it/s][32m2024-07-17 13:45:09.185[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 4.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 6.4289e-08, 'ids_predicted': 8.2703e-08}[0m
  8%|▊         | 29/385 [00:39<14:16,  2.41s/it][32m2024-07-17 13:45:10.620[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 4.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 3.3188e-07, 'ids_predicted': 2.1463e-07}[0m
  8%|▊         | 30/385 [00:40<12:30,  2.12s/it][32m2024-07-17 13:45:12.015[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 5.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 3.0687e-05, 'ids_predicted': 2.1441e-05}[0m
  8%|▊         | 31/385 [00:41<11:12,  1.90s/it][32m2024-07-17 13:45:12.785[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 5.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 6.3029e-10, 'ids_predicted': 1.2904e-09}[0m
  8%|▊         | 32/385 [00:42<09:10,  1.56s/it][32m2024-07-17 13:45:13.600[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 5.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 2.1636e-05, 'ids_predicted': 1.3988e-05}[0m
  9%|▊         | 33/385 [00:43<07:50,  1.34s/it][32m2024-07-17 13:45:15.459[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 5.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 2.7671e-06, 'ids_predicted': 2.3285e-07}[0m
  9%|▉         | 34/385 [00:45<08:44,  1.49s/it][32m2024-07-17 13:45:16.927[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 10.0, 'r_nw': 5.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 6.1037e-06, 'ids_predicted': 7.6211e-06}[0m
  9%|▉         | 35/385 [00:46<08:39,  1.49s/it][32m2024-07-17 13:45:19.085[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 2.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 7.6363e-07, 'ids_predicted': 1.4807e-06}[0m
  9%|▉         | 36/385 [00:48<09:48,  1.69s/it][32m2024-07-17 13:45:21.760[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 2.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.5822e-06, 'ids_predicted': 6.3916e-07}[0m
 10%|▉         | 37/385 [00:51<11:30,  1.98s/it][32m2024-07-17 13:45:22.670[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 2.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 6.6633e-07, 'ids_predicted': 4.9036e-07}[0m
 10%|▉         | 38/385 [00:52<09:36,  1.66s/it][32m2024-07-17 13:45:23.561[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 2.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 9.0945e-06, 'ids_predicted': 1.7471e-05}[0m
 10%|█         | 39/385 [00:53<08:14,  1.43s/it][32m2024-07-17 13:45:24.736[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 2.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.5139e-06, 'ids_predicted': 1.1499e-08}[0m
 10%|█         | 40/385 [00:54<07:47,  1.35s/it][32m2024-07-17 13:45:25.606[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 2.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 8.7154e-06, 'ids_predicted': 5.6714e-06}[0m
 11%|█         | 41/385 [00:55<06:55,  1.21s/it][32m2024-07-17 13:45:27.463[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 2.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 6.0573e-07, 'ids_predicted': 3.9262e-07}[0m
 11%|█         | 42/385 [00:57<08:01,  1.40s/it][32m2024-07-17 13:45:28.181[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 2.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 3.8706e-05, 'ids_predicted': 1.5576e-05}[0m
 11%|█         | 43/385 [00:57<06:49,  1.20s/it][32m2024-07-17 13:45:28.849[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 2.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.1885e-05, 'ids_predicted': 6.0875e-06}[0m
 11%|█▏        | 44/385 [00:58<05:54,  1.04s/it][32m2024-07-17 13:45:29.508[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 2.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 3.5101e-06, 'ids_predicted': 1.673e-06}[0m
 12%|█▏        | 45/385 [00:59<05:14,  1.08it/s][32m2024-07-17 13:45:30.299[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 3.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 5.594e-11, 'ids_predicted': 9.8796e-11}[0m
 12%|█▏        | 46/385 [01:00<04:59,  1.13it/s][32m2024-07-17 13:45:31.237[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 3.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 4.0238e-08, 'ids_predicted': 7.597e-09}[0m
 12%|█▏        | 47/385 [01:01<05:04,  1.11it/s][32m2024-07-17 13:45:33.110[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 3.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.1747e-05, 'ids_predicted': 3.1432e-05}[0m
 12%|█▏        | 48/385 [01:02<06:41,  1.19s/it][32m2024-07-17 13:45:34.236[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 3.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 3.1532e-11, 'ids_predicted': 5.6662e-10}[0m
 13%|█▎        | 49/385 [01:04<06:33,  1.17s/it][32m2024-07-17 13:45:34.974[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 3.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.5792e-09, 'ids_predicted': 1.8744e-07}[0m
 13%|█▎        | 50/385 [01:04<05:49,  1.04s/it][32m2024-07-17 13:45:35.677[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 3.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 9.6373e-06, 'ids_predicted': 2.7986e-05}[0m
 13%|█▎        | 51/385 [01:05<05:14,  1.06it/s][32m2024-07-17 13:45:38.363[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 3.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.0005e-08, 'ids_predicted': 1.4931e-08}[0m
 14%|█▎        | 52/385 [01:08<08:07,  1.46s/it][32m2024-07-17 13:45:39.148[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 3.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.103e-06, 'ids_predicted': 2.1618e-06}[0m
 14%|█▍        | 53/385 [01:08<06:58,  1.26s/it][32m2024-07-17 13:45:40.333[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 3.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.1343e-06, 'ids_predicted': 3.7727e-06}[0m
 14%|█▍        | 54/385 [01:10<06:49,  1.24s/it][32m2024-07-17 13:45:41.303[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 3.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 2.2663e-06, 'ids_predicted': 2.4777e-06}[0m
 14%|█▍        | 55/385 [01:11<06:22,  1.16s/it][32m2024-07-17 13:45:42.024[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 4.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.8428e-06, 'ids_predicted': 8.0999e-07}[0m
 15%|█▍        | 56/385 [01:11<05:37,  1.03s/it][32m2024-07-17 13:45:43.892[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 4.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.2194e-05, 'ids_predicted': 6.4997e-06}[0m
 15%|█▍        | 57/385 [01:13<06:59,  1.28s/it][32m2024-07-17 13:45:44.631[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 4.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.1884e-06, 'ids_predicted': 2.0422e-06}[0m
 15%|█▌        | 58/385 [01:14<06:05,  1.12s/it][32m2024-07-17 13:45:45.419[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 4.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 2.5424e-05, 'ids_predicted': 6.434e-06}[0m
 15%|█▌        | 59/385 [01:15<05:32,  1.02s/it][32m2024-07-17 13:45:47.204[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 4.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 5.4072e-06, 'ids_predicted': 5.8253e-06}[0m
 16%|█▌        | 60/385 [01:17<06:45,  1.25s/it][32m2024-07-17 13:45:48.068[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 4.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 3.5216e-06, 'ids_predicted': 6.2326e-06}[0m
 16%|█▌        | 61/385 [01:17<06:07,  1.13s/it][32m2024-07-17 13:45:49.431[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 4.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 6.4777e-06, 'ids_predicted': 9.9071e-06}[0m
 16%|█▌        | 62/385 [01:19<06:28,  1.20s/it][32m2024-07-17 13:45:50.066[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 4.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 7.6663e-06, 'ids_predicted': 1.2007e-05}[0m
 16%|█▋        | 63/385 [01:19<05:32,  1.03s/it][32m2024-07-17 13:45:50.827[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 4.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 3.0814e-05, 'ids_predicted': 1.617e-05}[0m
 17%|█▋        | 64/385 [01:20<05:05,  1.05it/s][32m2024-07-17 13:45:51.963[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 4.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.3576e-07, 'ids_predicted': 8.5346e-08}[0m
 17%|█▋        | 65/385 [01:21<05:21,  1.01s/it][32m2024-07-17 13:45:52.735[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 5.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.5504e-05, 'ids_predicted': 1.9358e-05}[0m
 17%|█▋        | 66/385 [01:22<04:58,  1.07it/s][32m2024-07-17 13:45:53.644[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 5.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 2.0617e-05, 'ids_predicted': 2.3731e-05}[0m
 17%|█▋        | 67/385 [01:23<04:55,  1.08it/s][32m2024-07-17 13:45:54.333[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 5.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 3.9108e-07, 'ids_predicted': 1.3035e-09}[0m
 18%|█▊        | 68/385 [01:24<04:31,  1.17it/s][32m2024-07-17 13:45:55.329[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 5.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.5315e-06, 'ids_predicted': 1.6457e-06}[0m
 18%|█▊        | 69/385 [01:25<04:43,  1.11it/s][32m2024-07-17 13:45:55.993[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 11.0, 'r_nw': 5.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 8.2666e-06, 'ids_predicted': 6.3774e-06}[0m
 18%|█▊        | 70/385 [01:25<04:20,  1.21it/s][32m2024-07-17 13:45:56.716[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 2.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 3.0256e-07, 'ids_predicted': 6.5743e-08}[0m
 18%|█▊        | 71/385 [01:26<04:10,  1.26it/s][32m2024-07-17 13:45:57.355[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 2.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.2164e-05, 'ids_predicted': 1.7858e-05}[0m
 19%|█▊        | 72/385 [01:27<03:54,  1.33it/s][32m2024-07-17 13:45:59.323[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 2.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.629e-05, 'ids_predicted': 1.3054e-05}[0m
 19%|█▉        | 73/385 [01:29<05:47,  1.12s/it][32m2024-07-17 13:45:59.998[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 2.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.786e-05, 'ids_predicted': 2.1684e-05}[0m
 19%|█▉        | 74/385 [01:29<05:05,  1.02it/s][32m2024-07-17 13:46:00.705[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 2.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 3.0301e-06, 'ids_predicted': 3.0879e-10}[0m
 19%|█▉        | 75/385 [01:30<04:39,  1.11it/s][32m2024-07-17 13:46:03.295[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 2.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 2.7689e-05, 'ids_predicted': 8.2845e-06}[0m
 20%|█▉        | 76/385 [01:33<07:14,  1.41s/it][32m2024-07-17 13:46:03.949[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 2.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.6451e-06, 'ids_predicted': 2.4116e-07}[0m
 20%|██        | 77/385 [01:33<06:03,  1.18s/it][32m2024-07-17 13:46:04.608[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 2.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 6.4244e-07, 'ids_predicted': 1.1692e-08}[0m
 20%|██        | 78/385 [01:34<05:14,  1.02s/it][32m2024-07-17 13:46:05.341[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 2.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 6.2751e-06, 'ids_predicted': 5.5588e-06}[0m
 21%|██        | 79/385 [01:35<04:46,  1.07it/s][32m2024-07-17 13:46:06.262[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 2.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 6.7829e-09, 'ids_predicted': 1.2502e-08}[0m
 21%|██        | 80/385 [01:36<04:44,  1.07it/s][32m2024-07-17 13:46:07.041[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 3.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 2.5835e-05, 'ids_predicted': 1.6252e-05}[0m
 21%|██        | 81/385 [01:36<04:29,  1.13it/s][32m2024-07-17 13:46:07.752[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 3.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 2.239e-05, 'ids_predicted': 1.0538e-05}[0m
 21%|██▏       | 82/385 [01:37<04:12,  1.20it/s][32m2024-07-17 13:46:09.412[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 3.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 3.4343e-05, 'ids_predicted': 6.5751e-06}[0m
 22%|██▏       | 83/385 [01:39<05:26,  1.08s/it][32m2024-07-17 13:46:11.749[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 3.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 8.4468e-06, 'ids_predicted': 8.9773e-06}[0m
 22%|██▏       | 84/385 [01:41<07:18,  1.46s/it][32m2024-07-17 13:46:12.484[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 3.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.3831e-05, 'ids_predicted': 1.5034e-05}[0m
 22%|██▏       | 85/385 [01:42<06:12,  1.24s/it][32m2024-07-17 13:46:17.509[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 3.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.2719e-10, 'ids_predicted': 4.9217e-08}[0m
 22%|██▏       | 86/385 [01:47<11:50,  2.38s/it][32m2024-07-17 13:46:18.181[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 3.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 8.7753e-06, 'ids_predicted': 1.5055e-05}[0m
 23%|██▎       | 87/385 [01:47<09:15,  1.86s/it][32m2024-07-17 13:46:18.860[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 3.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 3.8922e-06, 'ids_predicted': 9.0453e-06}[0m
 23%|██▎       | 88/385 [01:48<07:28,  1.51s/it][32m2024-07-17 13:46:19.486[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 3.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.6941e-09, 'ids_predicted': 2.7106e-07}[0m
 23%|██▎       | 89/385 [01:49<06:08,  1.24s/it][32m2024-07-17 13:46:20.225[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 3.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 7.4342e-09, 'ids_predicted': 6.7479e-09}[0m
 23%|██▎       | 90/385 [01:50<05:22,  1.09s/it][32m2024-07-17 13:46:20.869[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 4.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 5.0748e-11, 'ids_predicted': 5.0779e-11}[0m
 24%|██▎       | 91/385 [01:50<04:41,  1.04it/s][32m2024-07-17 13:46:21.857[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 4.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 2.2717e-10, 'ids_predicted': 1.531e-10}[0m
 24%|██▍       | 92/385 [01:51<04:43,  1.03it/s][32m2024-07-17 13:46:22.506[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 4.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 8.7012e-06, 'ids_predicted': 3.6058e-05}[0m
 24%|██▍       | 93/385 [01:52<04:14,  1.15it/s][32m2024-07-17 13:46:23.205[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 4.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 2.1944e-06, 'ids_predicted': 6.2074e-06}[0m
 24%|██▍       | 94/385 [01:53<03:58,  1.22it/s][32m2024-07-17 13:46:23.911[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 4.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 7.2972e-06, 'ids_predicted': 1.3989e-05}[0m
 25%|██▍       | 95/385 [01:53<03:47,  1.27it/s][32m2024-07-17 13:46:25.215[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 4.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 6.7853e-10, 'ids_predicted': 9.4773e-10}[0m
 25%|██▍       | 96/385 [01:55<04:32,  1.06it/s][32m2024-07-17 13:46:25.832[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 4.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 3.8333e-06, 'ids_predicted': 5.1285e-06}[0m
 25%|██▌       | 97/385 [01:55<04:03,  1.18it/s][32m2024-07-17 13:46:26.506[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 4.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 3.6998e-06, 'ids_predicted': 5.5273e-06}[0m
 25%|██▌       | 98/385 [01:56<03:47,  1.26it/s][32m2024-07-17 13:46:27.161[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 4.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 8.4113e-06, 'ids_predicted': 1.3579e-05}[0m
 26%|██▌       | 99/385 [01:56<03:34,  1.33it/s][32m2024-07-17 13:46:28.030[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 4.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 2.0885e-08, 'ids_predicted': 1.7764e-07}[0m
 26%|██▌       | 100/385 [01:57<03:44,  1.27it/s][32m2024-07-17 13:46:28.802[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 5.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 4.9689e-06, 'ids_predicted': 2.781e-06}[0m
 26%|██▌       | 101/385 [01:58<03:42,  1.28it/s][32m2024-07-17 13:46:29.389[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 5.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 4.3982e-07, 'ids_predicted': 3.4393e-06}[0m
 26%|██▋       | 102/385 [01:59<03:24,  1.38it/s][32m2024-07-17 13:46:30.296[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 5.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.6903e-08, 'ids_predicted': 6.1275e-07}[0m
 27%|██▋       | 103/385 [02:00<03:39,  1.28it/s][32m2024-07-17 13:46:31.081[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 5.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 2.5611e-05, 'ids_predicted': 3.1639e-05}[0m
 27%|██▋       | 104/385 [02:00<03:39,  1.28it/s][32m2024-07-17 13:46:31.965[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 12.0, 'r_nw': 5.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.7579e-05, 'ids_predicted': 7.0916e-06}[0m
 27%|██▋       | 105/385 [02:01<03:47,  1.23it/s][32m2024-07-17 13:46:32.672[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 2.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 6.8365e-11, 'ids_predicted': 1.4882e-09}[0m
 28%|██▊       | 106/385 [02:02<03:37,  1.28it/s][32m2024-07-17 13:46:33.843[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 2.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 2.2626e-08, 'ids_predicted': 9.0621e-09}[0m
 28%|██▊       | 107/385 [02:03<04:09,  1.11it/s][32m2024-07-17 13:46:34.683[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 2.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 6.2962e-09, 'ids_predicted': 8.1574e-09}[0m
 28%|██▊       | 108/385 [02:04<04:03,  1.14it/s][32m2024-07-17 13:46:35.582[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 2.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 8.9602e-06, 'ids_predicted': 7.7356e-06}[0m
 28%|██▊       | 109/385 [02:05<04:04,  1.13it/s][32m2024-07-17 13:46:36.407[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 2.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 8.2953e-06, 'ids_predicted': 5.0663e-06}[0m
 29%|██▊       | 110/385 [02:06<03:58,  1.15it/s][32m2024-07-17 13:46:37.353[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 2.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.6059e-05, 'ids_predicted': 1.4539e-05}[0m
 29%|██▉       | 111/385 [02:07<04:04,  1.12it/s][32m2024-07-17 13:46:38.025[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 2.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 4.1584e-11, 'ids_predicted': 2.7152e-10}[0m
 29%|██▉       | 112/385 [02:07<03:45,  1.21it/s][32m2024-07-17 13:46:38.663[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 2.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.3981e-05, 'ids_predicted': 1.5857e-05}[0m
 29%|██▉       | 113/385 [02:08<03:29,  1.30it/s][32m2024-07-17 13:46:39.381[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 2.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 5.7865e-06, 'ids_predicted': 3.2942e-06}[0m
 30%|██▉       | 114/385 [02:09<03:24,  1.33it/s][32m2024-07-17 13:46:42.146[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 2.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 3.1278e-05, 'ids_predicted': 2.1869e-05}[0m
 30%|██▉       | 115/385 [02:11<06:06,  1.36s/it][32m2024-07-17 13:46:42.850[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 3.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 3.148e-09, 'ids_predicted': 6.1409e-09}[0m
 30%|███       | 116/385 [02:12<05:12,  1.16s/it][32m2024-07-17 13:46:43.632[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 3.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.7092e-06, 'ids_predicted': 2.7095e-07}[0m
 30%|███       | 117/385 [02:13<04:40,  1.05s/it][32m2024-07-17 13:46:44.430[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 3.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 7.4584e-06, 'ids_predicted': 7.6379e-06}[0m
 31%|███       | 118/385 [02:14<04:19,  1.03it/s][32m2024-07-17 13:46:45.067[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 3.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 2.5906e-06, 'ids_predicted': 1.0362e-06}[0m
 31%|███       | 119/385 [02:14<03:51,  1.15it/s][32m2024-07-17 13:46:45.971[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 3.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 4.543e-05, 'ids_predicted': 9.9552e-06}[0m
 31%|███       | 120/385 [02:15<03:53,  1.13it/s][32m2024-07-17 13:46:48.348[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 3.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.522e-05, 'ids_predicted': 2.6863e-06}[0m
 31%|███▏      | 121/385 [02:18<05:51,  1.33s/it][32m2024-07-17 13:46:49.231[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 3.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.1602e-07, 'ids_predicted': 1.8904e-07}[0m
 32%|███▏      | 122/385 [02:19<05:14,  1.20s/it][32m2024-07-17 13:46:50.100[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 3.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 7.9054e-06, 'ids_predicted': 4.478e-06}[0m
 32%|███▏      | 123/385 [02:19<04:47,  1.10s/it][32m2024-07-17 13:46:50.757[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 3.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 2.8188e-07, 'ids_predicted': 9.0134e-08}[0m
 32%|███▏      | 124/385 [02:20<04:12,  1.04it/s][32m2024-07-17 13:46:51.395[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 3.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.3786e-06, 'ids_predicted': 3.5463e-07}[0m
 32%|███▏      | 125/385 [02:21<03:45,  1.15it/s][32m2024-07-17 13:46:52.098[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 4.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.1673e-05, 'ids_predicted': 1.4954e-05}[0m
 33%|███▎      | 126/385 [02:21<03:31,  1.22it/s][32m2024-07-17 13:46:52.852[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 4.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 7.497e-09, 'ids_predicted': 2.2035e-06}[0m
 33%|███▎      | 127/385 [02:22<03:26,  1.25it/s][32m2024-07-17 13:46:54.304[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 4.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 5.0469e-08, 'ids_predicted': 1.2668e-06}[0m
 33%|███▎      | 128/385 [02:24<04:15,  1.01it/s][32m2024-07-17 13:46:55.029[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 4.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 6.275e-06, 'ids_predicted': 6.3635e-06}[0m
 34%|███▎      | 129/385 [02:24<03:53,  1.09it/s][32m2024-07-17 13:46:55.747[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 4.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 5.5054e-06, 'ids_predicted': 9.8985e-06}[0m
 34%|███▍      | 130/385 [02:25<03:38,  1.17it/s][32m2024-07-17 13:46:56.664[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 4.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.9763e-05, 'ids_predicted': 2.216e-05}[0m
 34%|███▍      | 131/385 [02:26<03:41,  1.14it/s][32m2024-07-17 13:46:57.602[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 4.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.0677e-06, 'ids_predicted': 2.9658e-06}[0m
 34%|███▍      | 132/385 [02:27<03:45,  1.12it/s][32m2024-07-17 13:46:58.334[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 4.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 2.1902e-06, 'ids_predicted': 2.6177e-06}[0m
 35%|███▍      | 133/385 [02:28<03:32,  1.18it/s][32m2024-07-17 13:46:58.982[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 4.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 3.1589e-06, 'ids_predicted': 8.5238e-06}[0m
 35%|███▍      | 134/385 [02:28<03:17,  1.27it/s][32m2024-07-17 13:46:59.719[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 4.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.9649e-05, 'ids_predicted': 1.2628e-05}[0m
 35%|███▌      | 135/385 [02:29<03:12,  1.30it/s][32m2024-07-17 13:47:00.416[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 5.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.8986e-08, 'ids_predicted': 6.4048e-07}[0m
 35%|███▌      | 136/385 [02:30<03:06,  1.34it/s][32m2024-07-17 13:47:01.086[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 5.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.992e-06, 'ids_predicted': 3.7981e-06}[0m
 36%|███▌      | 137/385 [02:30<02:59,  1.38it/s][32m2024-07-17 13:47:03.821[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 5.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 2.078e-06, 'ids_predicted': 1.2382e-06}[0m
 36%|███▌      | 138/385 [02:33<05:28,  1.33s/it][32m2024-07-17 13:47:04.510[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 5.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.0516e-08, 'ids_predicted': 2.6385e-08}[0m
 36%|███▌      | 139/385 [02:34<04:39,  1.14s/it][32m2024-07-17 13:47:05.148[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 13.0, 'r_nw': 5.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 9.8983e-07, 'ids_predicted': 2.3519e-06}[0m
 36%|███▋      | 140/385 [02:34<04:01,  1.01it/s][32m2024-07-17 13:47:07.834[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 2.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.9455e-06, 'ids_predicted': 2.4668e-06}[0m
 37%|███▋      | 141/385 [02:37<06:05,  1.50s/it][32m2024-07-17 13:47:08.720[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 2.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 9.9498e-08, 'ids_predicted': 4.0726e-07}[0m
 37%|███▋      | 142/385 [02:38<05:19,  1.31s/it][32m2024-07-17 13:47:09.797[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 2.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.3022e-07, 'ids_predicted': 9.5106e-07}[0m
 37%|███▋      | 143/385 [02:39<05:00,  1.24s/it][32m2024-07-17 13:47:10.469[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 2.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.5253e-05, 'ids_predicted': 1.7495e-05}[0m
 37%|███▋      | 144/385 [02:40<04:18,  1.07s/it][32m2024-07-17 13:47:11.169[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 2.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 2.8079e-06, 'ids_predicted': 5.2798e-06}[0m
 38%|███▊      | 145/385 [02:40<03:50,  1.04it/s][32m2024-07-17 13:47:11.992[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 2.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.8563e-08, 'ids_predicted': 6.2768e-09}[0m
 38%|███▊      | 146/385 [02:41<03:39,  1.09it/s][32m2024-07-17 13:47:12.700[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 2.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 7.4137e-06, 'ids_predicted': 9.8175e-06}[0m
 38%|███▊      | 147/385 [02:42<03:23,  1.17it/s][32m2024-07-17 13:47:13.396[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 2.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 8.0526e-06, 'ids_predicted': 1.9982e-05}[0m
 38%|███▊      | 148/385 [02:43<03:11,  1.24it/s][32m2024-07-17 13:47:14.263[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 2.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 4.1001e-08, 'ids_predicted': 1.0523e-08}[0m
 39%|███▊      | 149/385 [02:44<03:14,  1.21it/s][32m2024-07-17 13:47:15.355[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 2.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.27e-05, 'ids_predicted': 1.0814e-05}[0m
 39%|███▉      | 150/385 [02:45<03:32,  1.10it/s][32m2024-07-17 13:47:16.089[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 3.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 3.5855e-07, 'ids_predicted': 4.4055e-07}[0m
 39%|███▉      | 151/385 [02:45<03:19,  1.17it/s][32m2024-07-17 13:47:16.958[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 3.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.1958e-05, 'ids_predicted': 1.732e-05}[0m
 39%|███▉      | 152/385 [02:46<03:20,  1.16it/s][32m2024-07-17 13:47:17.619[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 3.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 2.8344e-06, 'ids_predicted': 1.2183e-06}[0m
 40%|███▉      | 153/385 [02:47<03:05,  1.25it/s][32m2024-07-17 13:47:18.284[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 3.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 2.9581e-06, 'ids_predicted': 2.7273e-06}[0m
 40%|████      | 154/385 [02:48<02:55,  1.32it/s][32m2024-07-17 13:47:19.189[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 3.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 2.3697e-05, 'ids_predicted': 1.3701e-05}[0m
 40%|████      | 155/385 [02:49<03:04,  1.25it/s][32m2024-07-17 13:47:19.893[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 3.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 2.523e-08, 'ids_predicted': 2.4344e-10}[0m
 41%|████      | 156/385 [02:49<02:56,  1.29it/s][32m2024-07-17 13:47:20.626[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 3.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 5.7961e-07, 'ids_predicted': 5.0437e-09}[0m
 41%|████      | 157/385 [02:50<02:53,  1.31it/s][32m2024-07-17 13:47:21.311[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 3.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.9917e-05, 'ids_predicted': 1.7085e-05}[0m
 41%|████      | 158/385 [02:51<02:47,  1.35it/s][32m2024-07-17 13:47:23.633[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 3.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 3.3603e-06, 'ids_predicted': 2.4712e-06}[0m
 41%|████▏     | 159/385 [02:53<04:34,  1.21s/it][32m2024-07-17 13:47:24.357[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 3.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 3.8197e-05, 'ids_predicted': 9.5254e-06}[0m
 42%|████▏     | 160/385 [02:54<04:00,  1.07s/it][32m2024-07-17 13:47:25.286[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 4.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 3.0061e-07, 'ids_predicted': 7.1497e-08}[0m
 42%|████▏     | 161/385 [02:55<03:49,  1.03s/it][32m2024-07-17 13:47:26.003[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 4.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.4531e-05, 'ids_predicted': 9.0974e-06}[0m
 42%|████▏     | 162/385 [02:55<03:27,  1.07it/s][32m2024-07-17 13:47:26.675[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 4.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.4766e-08, 'ids_predicted': 2.5568e-09}[0m
 42%|████▏     | 163/385 [02:56<03:09,  1.17it/s][32m2024-07-17 13:47:27.399[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 4.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 6.5728e-10, 'ids_predicted': 6.7903e-07}[0m
 43%|████▎     | 164/385 [02:57<03:00,  1.23it/s][32m2024-07-17 13:47:28.097[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 4.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.1806e-05, 'ids_predicted': 8.168e-06}[0m
 43%|████▎     | 165/385 [02:57<02:51,  1.28it/s][32m2024-07-17 13:47:28.757[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 4.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 9.1379e-06, 'ids_predicted': 2.7512e-05}[0m
 43%|████▎     | 166/385 [02:58<02:42,  1.34it/s][32m2024-07-17 13:47:29.408[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 4.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.5344e-06, 'ids_predicted': 7.7866e-06}[0m
 43%|████▎     | 167/385 [02:59<02:36,  1.40it/s][32m2024-07-17 13:47:30.024[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 4.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 5.5452e-06, 'ids_predicted': 2.3463e-06}[0m
 44%|████▎     | 168/385 [02:59<02:28,  1.46it/s][32m2024-07-17 13:47:30.712[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 4.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 7.576e-07, 'ids_predicted': 3.4321e-06}[0m
 44%|████▍     | 169/385 [03:00<02:28,  1.46it/s][32m2024-07-17 13:47:31.399[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 4.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.9749e-10, 'ids_predicted': 3.568e-10}[0m
 44%|████▍     | 170/385 [03:01<02:27,  1.46it/s][32m2024-07-17 13:47:32.135[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 5.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.3887e-09, 'ids_predicted': 3.7071e-07}[0m
 44%|████▍     | 171/385 [03:01<02:30,  1.43it/s][32m2024-07-17 13:47:32.798[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 5.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.5208e-05, 'ids_predicted': 7.3307e-06}[0m
 45%|████▍     | 172/385 [03:02<02:26,  1.45it/s][32m2024-07-17 13:47:33.699[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 5.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 8.044e-08, 'ids_predicted': 1.8545e-06}[0m
 45%|████▍     | 173/385 [03:03<02:39,  1.33it/s][32m2024-07-17 13:47:35.125[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 5.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 3.2362e-09, 'ids_predicted': 9.1834e-08}[0m
 45%|████▌     | 174/385 [03:04<03:21,  1.05it/s][32m2024-07-17 13:47:35.865[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 14.0, 'r_nw': 5.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 2.0166e-07, 'ids_predicted': 1.1836e-09}[0m
 45%|████▌     | 175/385 [03:05<03:06,  1.12it/s][32m2024-07-17 13:47:36.575[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 2.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 2.0277e-06, 'ids_predicted': 3.4629e-06}[0m
 46%|████▌     | 176/385 [03:06<02:54,  1.20it/s][32m2024-07-17 13:47:37.540[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 2.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 5.8143e-08, 'ids_predicted': 1.5572e-09}[0m
 46%|████▌     | 177/385 [03:07<03:01,  1.14it/s][32m2024-07-17 13:47:38.306[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 2.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.6636e-05, 'ids_predicted': 2.5116e-05}[0m
 46%|████▌     | 178/385 [03:08<02:54,  1.19it/s][32m2024-07-17 13:47:38.967[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 2.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 2.0276e-05, 'ids_predicted': 9.8075e-06}[0m
 46%|████▋     | 179/385 [03:08<02:42,  1.27it/s][32m2024-07-17 13:47:39.675[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 2.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 9.1139e-07, 'ids_predicted': 6.4481e-09}[0m
 47%|████▋     | 180/385 [03:09<02:36,  1.31it/s][32m2024-07-17 13:47:40.360[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 2.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.0474e-05, 'ids_predicted': 8.9587e-06}[0m
 47%|████▋     | 181/385 [03:10<02:31,  1.35it/s][32m2024-07-17 13:47:41.022[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 2.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 4.5789e-07, 'ids_predicted': 1.6563e-08}[0m
 47%|████▋     | 182/385 [03:10<02:25,  1.40it/s][32m2024-07-17 13:47:41.664[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 2.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 5.1422e-07, 'ids_predicted': 3.2024e-06}[0m
 48%|████▊     | 183/385 [03:11<02:20,  1.44it/s][32m2024-07-17 13:47:42.574[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 2.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 4.0983e-06, 'ids_predicted': 6.8334e-06}[0m
 48%|████▊     | 184/385 [03:12<02:32,  1.32it/s][32m2024-07-17 13:47:43.215[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 2.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 2.1995e-07, 'ids_predicted': 9.4824e-09}[0m
 48%|████▊     | 185/385 [03:13<02:24,  1.38it/s][32m2024-07-17 13:47:43.941[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 3.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 9.5765e-06, 'ids_predicted': 6.8983e-06}[0m
 48%|████▊     | 186/385 [03:13<02:24,  1.38it/s][32m2024-07-17 13:47:44.624[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 3.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 2.2972e-05, 'ids_predicted': 1.7681e-05}[0m
 49%|████▊     | 187/385 [03:14<02:20,  1.40it/s][32m2024-07-17 13:47:45.314[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 3.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.4812e-05, 'ids_predicted': 7.688e-06}[0m
 49%|████▉     | 188/385 [03:15<02:18,  1.42it/s][32m2024-07-17 13:47:46.065[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 3.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 6.0821e-06, 'ids_predicted': 9.2698e-06}[0m
 49%|████▉     | 189/385 [03:15<02:20,  1.39it/s][32m2024-07-17 13:47:46.825[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 3.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.5033e-05, 'ids_predicted': 1.197e-05}[0m
 49%|████▉     | 190/385 [03:16<02:22,  1.37it/s][32m2024-07-17 13:47:47.760[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 3.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.0629e-05, 'ids_predicted': 1.4279e-05}[0m
 50%|████▉     | 191/385 [03:17<02:33,  1.26it/s][32m2024-07-17 13:47:48.479[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 3.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.1259e-05, 'ids_predicted': 6.8979e-06}[0m
 50%|████▉     | 192/385 [03:18<02:28,  1.30it/s][32m2024-07-17 13:47:49.187[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 3.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 2.7766e-06, 'ids_predicted': 1.599e-06}[0m
 50%|█████     | 193/385 [03:19<02:24,  1.33it/s][32m2024-07-17 13:47:49.917[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 3.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.105e-05, 'ids_predicted': 9.0923e-06}[0m
 50%|█████     | 194/385 [03:19<02:22,  1.34it/s][32m2024-07-17 13:47:50.671[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 3.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 6.5995e-06, 'ids_predicted': 6.2758e-06}[0m
 51%|█████     | 195/385 [03:20<02:22,  1.34it/s][32m2024-07-17 13:47:51.499[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 4.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 3.7994e-06, 'ids_predicted': 2.3828e-06}[0m
 51%|█████     | 196/385 [03:21<02:25,  1.30it/s][32m2024-07-17 13:47:52.428[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 4.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.0641e-05, 'ids_predicted': 7.7402e-06}[0m
 51%|█████     | 197/385 [03:22<02:33,  1.22it/s][32m2024-07-17 13:47:53.108[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 4.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 5.6013e-06, 'ids_predicted': 2.9923e-06}[0m
 51%|█████▏    | 198/385 [03:22<02:25,  1.29it/s][32m2024-07-17 13:47:53.809[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 4.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.8627e-06, 'ids_predicted': 2.9726e-07}[0m
 52%|█████▏    | 199/385 [03:23<02:20,  1.33it/s][32m2024-07-17 13:47:54.484[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 4.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 2.3654e-05, 'ids_predicted': 1.0462e-05}[0m
 52%|█████▏    | 200/385 [03:24<02:15,  1.37it/s][32m2024-07-17 13:47:55.309[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 4.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 4.8046e-09, 'ids_predicted': 3.5854e-09}[0m
 52%|█████▏    | 201/385 [03:25<02:19,  1.32it/s][32m2024-07-17 13:47:56.217[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 4.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 6.6664e-06, 'ids_predicted': 3.7753e-06}[0m
 52%|█████▏    | 202/385 [03:26<02:27,  1.24it/s][32m2024-07-17 13:47:56.908[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 4.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 5.1492e-09, 'ids_predicted': 4.1293e-07}[0m
 53%|█████▎    | 203/385 [03:26<02:20,  1.30it/s][32m2024-07-17 13:47:57.595[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 4.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 7.1067e-06, 'ids_predicted': 1.9558e-05}[0m
 53%|█████▎    | 204/385 [03:27<02:14,  1.34it/s][32m2024-07-17 13:47:58.553[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 4.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 3.317e-06, 'ids_predicted': 9.811e-06}[0m
 53%|█████▎    | 205/385 [03:28<02:25,  1.24it/s][32m2024-07-17 13:47:59.418[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 5.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 2.3658e-13, 'ids_predicted': 1.515e-10}[0m
 54%|█████▎    | 206/385 [03:29<02:27,  1.21it/s][32m2024-07-17 13:48:00.584[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 5.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.5071e-09, 'ids_predicted': 6.9316e-10}[0m
 54%|█████▍    | 207/385 [03:30<02:45,  1.08it/s][32m2024-07-17 13:48:01.307[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 5.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 2.7673e-07, 'ids_predicted': 1.6794e-06}[0m
 54%|█████▍    | 208/385 [03:31<02:33,  1.15it/s][32m2024-07-17 13:48:02.205[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 5.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.6008e-09, 'ids_predicted': 2.5502e-08}[0m
 54%|█████▍    | 209/385 [03:32<02:34,  1.14it/s][32m2024-07-17 13:48:03.428[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 15.0, 'r_nw': 5.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 3.0911e-10, 'ids_predicted': 6.5179e-10}[0m
 55%|█████▍    | 210/385 [03:33<02:51,  1.02it/s][32m2024-07-17 13:48:04.100[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 2.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 3.2188e-10, 'ids_predicted': 6.3475e-10}[0m
 55%|█████▍    | 211/385 [03:33<02:34,  1.13it/s][32m2024-07-17 13:48:04.823[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 2.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 6.5563e-06, 'ids_predicted': 4.5353e-06}[0m
 55%|█████▌    | 212/385 [03:34<02:25,  1.19it/s][32m2024-07-17 13:48:05.502[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 2.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 8.247e-08, 'ids_predicted': 5.1312e-07}[0m
 55%|█████▌    | 213/385 [03:35<02:15,  1.27it/s][32m2024-07-17 13:48:06.271[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 2.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.0759e-05, 'ids_predicted': 9.1696e-06}[0m
 56%|█████▌    | 214/385 [03:36<02:14,  1.28it/s][32m2024-07-17 13:48:07.032[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 2.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 8.2328e-07, 'ids_predicted': 1.7753e-06}[0m
 56%|█████▌    | 215/385 [03:36<02:12,  1.29it/s][32m2024-07-17 13:48:07.855[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 2.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.6056e-09, 'ids_predicted': 3.1869e-10}[0m
 56%|█████▌    | 216/385 [03:37<02:13,  1.26it/s][32m2024-07-17 13:48:08.537[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 2.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 3.1669e-06, 'ids_predicted': 3.5922e-06}[0m
 56%|█████▋    | 217/385 [03:38<02:07,  1.32it/s][32m2024-07-17 13:48:09.347[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 2.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.1839e-08, 'ids_predicted': 3.686e-08}[0m
 57%|█████▋    | 218/385 [03:39<02:09,  1.29it/s][32m2024-07-17 13:48:10.218[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 2.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.3006e-06, 'ids_predicted': 1.6819e-06}[0m
 57%|█████▋    | 219/385 [03:40<02:13,  1.25it/s][32m2024-07-17 13:48:10.875[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 2.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 2.5527e-06, 'ids_predicted': 3.1282e-06}[0m
 57%|█████▋    | 220/385 [03:40<02:05,  1.32it/s][32m2024-07-17 13:48:11.621[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 3.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 2.6516e-06, 'ids_predicted': 3.9831e-06}[0m
 57%|█████▋    | 221/385 [03:41<02:03,  1.32it/s][32m2024-07-17 13:48:12.302[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 3.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 2.5339e-06, 'ids_predicted': 5.2723e-07}[0m
 58%|█████▊    | 222/385 [03:42<01:59,  1.36it/s][32m2024-07-17 13:48:12.979[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 3.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.6977e-08, 'ids_predicted': 1.5923e-08}[0m
 58%|█████▊    | 223/385 [03:42<01:55,  1.40it/s][32m2024-07-17 13:48:13.684[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 3.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.3149e-05, 'ids_predicted': 8.4829e-06}[0m
 58%|█████▊    | 224/385 [03:43<01:54,  1.40it/s][32m2024-07-17 13:48:14.384[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 3.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.4332e-05, 'ids_predicted': 1.7154e-05}[0m
 58%|█████▊    | 225/385 [03:44<01:53,  1.41it/s][32m2024-07-17 13:48:15.026[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 3.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 3.9332e-06, 'ids_predicted': 1.851e-06}[0m
 59%|█████▊    | 226/385 [03:44<01:49,  1.45it/s][32m2024-07-17 13:48:15.924[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 3.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.9977e-05, 'ids_predicted': 6.4256e-06}[0m
 59%|█████▉    | 227/385 [03:45<01:58,  1.33it/s][32m2024-07-17 13:48:16.591[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 3.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 9.801e-06, 'ids_predicted': 9.1562e-06}[0m
 59%|█████▉    | 228/385 [03:46<01:54,  1.38it/s][32m2024-07-17 13:48:17.324[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 3.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 6.9316e-06, 'ids_predicted': 4.7821e-06}[0m
 59%|█████▉    | 229/385 [03:47<01:53,  1.37it/s][32m2024-07-17 13:48:17.983[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 3.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 2.914e-07, 'ids_predicted': 3.046e-09}[0m
 60%|█████▉    | 230/385 [03:47<01:49,  1.41it/s][32m2024-07-17 13:48:18.702[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 4.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 2.5892e-06, 'ids_predicted': 2.4682e-06}[0m
 60%|██████    | 231/385 [03:48<01:49,  1.41it/s][32m2024-07-17 13:48:19.429[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 4.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.3302e-09, 'ids_predicted': 3.0433e-09}[0m
 60%|██████    | 232/385 [03:49<01:49,  1.40it/s][32m2024-07-17 13:48:20.072[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 4.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 2.3162e-05, 'ids_predicted': 2.1683e-05}[0m
 61%|██████    | 233/385 [03:49<01:45,  1.44it/s][32m2024-07-17 13:48:21.000[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 4.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 6.6331e-09, 'ids_predicted': 2.0547e-09}[0m
 61%|██████    | 234/385 [03:50<01:55,  1.31it/s][32m2024-07-17 13:48:21.715[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 4.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 4.3046e-05, 'ids_predicted': 3.4923e-05}[0m
 61%|██████    | 235/385 [03:51<01:52,  1.33it/s][32m2024-07-17 13:48:22.366[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 4.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 9.8494e-06, 'ids_predicted': 8.5475e-06}[0m
 61%|██████▏   | 236/385 [03:52<01:47,  1.39it/s][32m2024-07-17 13:48:23.089[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 4.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.1577e-06, 'ids_predicted': 7.6683e-09}[0m
 62%|██████▏   | 237/385 [03:52<01:46,  1.39it/s][32m2024-07-17 13:48:24.005[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 4.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 2.9619e-06, 'ids_predicted': 4.9451e-06}[0m
 62%|██████▏   | 238/385 [03:53<01:54,  1.28it/s][32m2024-07-17 13:48:24.657[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 4.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.9516e-05, 'ids_predicted': 1.5616e-05}[0m
 62%|██████▏   | 239/385 [03:54<01:48,  1.35it/s][32m2024-07-17 13:48:25.439[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 4.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 3.1498e-07, 'ids_predicted': 4.8446e-07}[0m
 62%|██████▏   | 240/385 [03:55<01:49,  1.33it/s][32m2024-07-17 13:48:26.168[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 5.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 6.617e-08, 'ids_predicted': 5.8155e-10}[0m
 63%|██████▎   | 241/385 [03:55<01:47,  1.34it/s][32m2024-07-17 13:48:26.935[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 5.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.1124e-10, 'ids_predicted': 2.3965e-09}[0m
 63%|██████▎   | 242/385 [03:56<01:47,  1.33it/s][32m2024-07-17 13:48:27.818[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 5.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 8.6128e-06, 'ids_predicted': 1.2083e-05}[0m
 63%|██████▎   | 243/385 [03:57<01:52,  1.26it/s][32m2024-07-17 13:48:28.570[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 5.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 6.298e-06, 'ids_predicted': 7.6594e-06}[0m
 63%|██████▎   | 244/385 [03:58<01:49,  1.28it/s][32m2024-07-17 13:48:29.199[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 16.0, 'r_nw': 5.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 9.2809e-06, 'ids_predicted': 3.5503e-05}[0m
 64%|██████▎   | 245/385 [03:59<01:42,  1.36it/s][32m2024-07-17 13:48:29.824[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 2.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 5.5854e-06, 'ids_predicted': 6.0989e-06}[0m
 64%|██████▍   | 246/385 [03:59<01:37,  1.43it/s][32m2024-07-17 13:48:30.478[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 2.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 6.5381e-06, 'ids_predicted': 1.8493e-05}[0m
 64%|██████▍   | 247/385 [04:00<01:34,  1.46it/s][32m2024-07-17 13:48:31.368[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 2.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 7.8534e-06, 'ids_predicted': 1.2963e-05}[0m
 64%|██████▍   | 248/385 [04:01<01:42,  1.34it/s][32m2024-07-17 13:48:32.041[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 2.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 2.1686e-06, 'ids_predicted': 6.6991e-06}[0m
 65%|██████▍   | 249/385 [04:01<01:38,  1.38it/s][32m2024-07-17 13:48:32.699[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 2.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 2.6695e-10, 'ids_predicted': 1.1799e-10}[0m
 65%|██████▍   | 250/385 [04:02<01:35,  1.42it/s][32m2024-07-17 13:48:33.394[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 2.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 2.8765e-10, 'ids_predicted': 1.6715e-10}[0m
 65%|██████▌   | 251/385 [04:03<01:34,  1.42it/s][32m2024-07-17 13:48:34.096[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 2.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.2298e-05, 'ids_predicted': 6.8824e-06}[0m
 65%|██████▌   | 252/385 [04:03<01:33,  1.42it/s][32m2024-07-17 13:48:34.774[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 2.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.0664e-05, 'ids_predicted': 1.2002e-05}[0m
 66%|██████▌   | 253/385 [04:04<01:31,  1.44it/s][32m2024-07-17 13:48:35.414[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 2.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 9.3948e-06, 'ids_predicted': 1.2809e-05}[0m
 66%|██████▌   | 254/385 [04:05<01:28,  1.47it/s][32m2024-07-17 13:48:36.176[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 2.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 9.7097e-06, 'ids_predicted': 1.6362e-05}[0m
 66%|██████▌   | 255/385 [04:05<01:31,  1.42it/s][32m2024-07-17 13:48:36.901[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 3.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.133e-05, 'ids_predicted': 1.1415e-05}[0m
 66%|██████▋   | 256/385 [04:06<01:31,  1.41it/s][32m2024-07-17 13:48:37.561[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 3.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 3.092e-05, 'ids_predicted': 1.4985e-05}[0m
 67%|██████▋   | 257/385 [04:07<01:28,  1.44it/s][32m2024-07-17 13:48:38.216[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 3.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 2.7997e-06, 'ids_predicted': 6.5458e-06}[0m
 67%|██████▋   | 258/385 [04:08<01:26,  1.46it/s][32m2024-07-17 13:48:38.927[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 3.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.7968e-09, 'ids_predicted': 6.6079e-11}[0m
 67%|██████▋   | 259/385 [04:08<01:27,  1.45it/s][32m2024-07-17 13:48:39.675[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 3.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.4206e-05, 'ids_predicted': 2.3821e-05}[0m
 68%|██████▊   | 260/385 [04:09<01:28,  1.41it/s][32m2024-07-17 13:48:40.425[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 3.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 2.7151e-06, 'ids_predicted': 1.1144e-06}[0m
 68%|██████▊   | 261/385 [04:10<01:29,  1.39it/s][32m2024-07-17 13:48:41.123[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 3.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 9.577e-06, 'ids_predicted': 1.3975e-05}[0m
 68%|██████▊   | 262/385 [04:10<01:27,  1.40it/s][32m2024-07-17 13:48:41.958[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 3.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 3.9417e-06, 'ids_predicted': 2.8494e-06}[0m
 68%|██████▊   | 263/385 [04:11<01:31,  1.33it/s][32m2024-07-17 13:48:42.860[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 3.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 5.6874e-06, 'ids_predicted': 2.0459e-06}[0m
 69%|██████▊   | 264/385 [04:12<01:36,  1.26it/s][32m2024-07-17 13:48:43.561[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 3.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 2.6155e-05, 'ids_predicted': 1.8629e-05}[0m
 69%|██████▉   | 265/385 [04:13<01:32,  1.30it/s][32m2024-07-17 13:48:44.207[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 4.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 2.0136e-05, 'ids_predicted': 2.5544e-05}[0m
 69%|██████▉   | 266/385 [04:14<01:26,  1.37it/s][32m2024-07-17 13:48:44.897[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 4.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.3546e-05, 'ids_predicted': 8.0259e-06}[0m
 69%|██████▉   | 267/385 [04:14<01:24,  1.39it/s][32m2024-07-17 13:48:45.583[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 4.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 3.5359e-05, 'ids_predicted': 2.3993e-05}[0m
 70%|██████▉   | 268/385 [04:15<01:22,  1.41it/s][32m2024-07-17 13:48:46.468[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 4.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.4971e-08, 'ids_predicted': 1.7595e-08}[0m
 70%|██████▉   | 269/385 [04:16<01:28,  1.31it/s][32m2024-07-17 13:48:47.203[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 4.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.46e-05, 'ids_predicted': 8.0271e-06}[0m
 70%|███████   | 270/385 [04:17<01:26,  1.33it/s][32m2024-07-17 13:48:47.889[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 4.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.9471e-05, 'ids_predicted': 1.0479e-05}[0m
 70%|███████   | 271/385 [04:17<01:23,  1.36it/s][32m2024-07-17 13:48:48.771[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 4.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 9.7826e-06, 'ids_predicted': 6.2333e-06}[0m
 71%|███████   | 272/385 [04:18<01:27,  1.29it/s][32m2024-07-17 13:48:49.496[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 4.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 9.8748e-06, 'ids_predicted': 1.5317e-05}[0m
 71%|███████   | 273/385 [04:19<01:25,  1.31it/s][32m2024-07-17 13:48:50.205[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 4.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 5.8815e-06, 'ids_predicted': 3.0781e-06}[0m
 71%|███████   | 274/385 [04:20<01:22,  1.34it/s][32m2024-07-17 13:48:50.920[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 4.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 2.5545e-05, 'ids_predicted': 8.2503e-06}[0m
 71%|███████▏  | 275/385 [04:20<01:21,  1.36it/s][32m2024-07-17 13:48:51.616[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 5.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.4017e-05, 'ids_predicted': 1.4339e-05}[0m
 72%|███████▏  | 276/385 [04:21<01:18,  1.38it/s][32m2024-07-17 13:48:52.311[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 5.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.1878e-05, 'ids_predicted': 1.8328e-05}[0m
 72%|███████▏  | 277/385 [04:22<01:17,  1.40it/s][32m2024-07-17 13:48:52.978[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 5.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 3.7668e-09, 'ids_predicted': 5.7109e-10}[0m
 72%|███████▏  | 278/385 [04:22<01:15,  1.43it/s][32m2024-07-17 13:48:53.737[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 5.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 8.6304e-06, 'ids_predicted': 2.5295e-06}[0m
 72%|███████▏  | 279/385 [04:23<01:16,  1.39it/s][32m2024-07-17 13:48:54.359[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 17.0, 'r_nw': 5.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 6.9443e-06, 'ids_predicted': 3.6661e-06}[0m
 73%|███████▎  | 280/385 [04:24<01:12,  1.45it/s][32m2024-07-17 13:48:55.023[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 2.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.6486e-06, 'ids_predicted': 6.3804e-06}[0m
 73%|███████▎  | 281/385 [04:24<01:10,  1.47it/s][32m2024-07-17 13:48:55.732[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 2.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 7.5594e-07, 'ids_predicted': 2.9589e-06}[0m
 73%|███████▎  | 282/385 [04:25<01:11,  1.45it/s][32m2024-07-17 13:48:56.372[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 2.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 4.6185e-06, 'ids_predicted': 1.4349e-05}[0m
 74%|███████▎  | 283/385 [04:26<01:08,  1.48it/s][32m2024-07-17 13:48:57.134[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 2.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 6.1525e-06, 'ids_predicted': 2.0786e-05}[0m
 74%|███████▍  | 284/385 [04:26<01:10,  1.43it/s][32m2024-07-17 13:48:57.801[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 2.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 7.6117e-06, 'ids_predicted': 1.2689e-05}[0m
 74%|███████▍  | 285/385 [04:27<01:09,  1.45it/s][32m2024-07-17 13:48:58.684[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 2.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 3.7294e-06, 'ids_predicted': 3.3697e-06}[0m
 74%|███████▍  | 286/385 [04:28<01:14,  1.34it/s][32m2024-07-17 13:48:59.381[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 2.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 8.8027e-13, 'ids_predicted': 3.775e-10}[0m
 75%|███████▍  | 287/385 [04:29<01:11,  1.36it/s][32m2024-07-17 13:49:00.300[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 2.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.8621e-06, 'ids_predicted': 3.9898e-06}[0m
 75%|███████▍  | 288/385 [04:30<01:16,  1.27it/s][32m2024-07-17 13:49:00.996[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 2.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 4.3812e-06, 'ids_predicted': 1.3491e-05}[0m
 75%|███████▌  | 289/385 [04:30<01:13,  1.31it/s][32m2024-07-17 13:49:01.649[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 2.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 2.3851e-07, 'ids_predicted': 1.4637e-06}[0m
 75%|███████▌  | 290/385 [04:31<01:09,  1.37it/s][32m2024-07-17 13:49:02.336[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 3.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 8.3949e-06, 'ids_predicted': 2.3782e-05}[0m
 76%|███████▌  | 291/385 [04:32<01:07,  1.40it/s][32m2024-07-17 13:49:03.009[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 3.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.064e-05, 'ids_predicted': 1.6197e-05}[0m
 76%|███████▌  | 292/385 [04:32<01:05,  1.42it/s][32m2024-07-17 13:49:03.716[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 3.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.6174e-05, 'ids_predicted': 2.7382e-05}[0m
 76%|███████▌  | 293/385 [04:33<01:04,  1.42it/s][32m2024-07-17 13:49:04.379[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 3.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 9.5977e-10, 'ids_predicted': 9.9444e-10}[0m
 76%|███████▋  | 294/385 [04:34<01:02,  1.44it/s][32m2024-07-17 13:49:05.328[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 3.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.4438e-05, 'ids_predicted': 1.8644e-05}[0m
 77%|███████▋  | 295/385 [04:35<01:09,  1.30it/s][32m2024-07-17 13:49:06.000[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 3.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.034e-05, 'ids_predicted': 1.5502e-05}[0m
 77%|███████▋  | 296/385 [04:35<01:05,  1.35it/s][32m2024-07-17 13:49:06.682[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 3.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 2.7983e-05, 'ids_predicted': 9.6066e-06}[0m
 77%|███████▋  | 297/385 [04:36<01:03,  1.38it/s][32m2024-07-17 13:49:07.353[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 3.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.6783e-05, 'ids_predicted': 8.8479e-06}[0m
 77%|███████▋  | 298/385 [04:37<01:01,  1.41it/s][32m2024-07-17 13:49:08.048[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 3.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.7375e-05, 'ids_predicted': 2.4194e-05}[0m
 78%|███████▊  | 299/385 [04:37<01:00,  1.42it/s][32m2024-07-17 13:49:11.354[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 3.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 7.4828e-06, 'ids_predicted': 2.6189e-06}[0m
 78%|███████▊  | 300/385 [04:41<02:06,  1.48s/it][32m2024-07-17 13:49:12.253[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 4.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.049e-05, 'ids_predicted': 3.0521e-05}[0m
 78%|███████▊  | 301/385 [04:42<01:49,  1.31s/it][32m2024-07-17 13:49:13.201[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 4.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 3.8031e-06, 'ids_predicted': 4.5682e-06}[0m
 78%|███████▊  | 302/385 [04:43<01:39,  1.20s/it][32m2024-07-17 13:49:13.893[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 4.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 6.7482e-09, 'ids_predicted': 1.7052e-09}[0m
 79%|███████▊  | 303/385 [04:43<01:25,  1.05s/it][32m2024-07-17 13:49:14.874[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 4.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 2.625e-05, 'ids_predicted': 1.4585e-05}[0m
 79%|███████▉  | 304/385 [04:44<01:23,  1.03s/it][32m2024-07-17 13:49:15.704[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 4.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 4.7418e-06, 'ids_predicted': 5.7762e-06}[0m
 79%|███████▉  | 305/385 [04:45<01:17,  1.03it/s][32m2024-07-17 13:49:16.603[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 4.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 2.0856e-05, 'ids_predicted': 2.0716e-05}[0m
 79%|███████▉  | 306/385 [04:46<01:14,  1.06it/s][32m2024-07-17 13:49:17.558[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 4.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 7.8235e-09, 'ids_predicted': 8.4533e-10}[0m
 80%|███████▉  | 307/385 [04:47<01:14,  1.05it/s][32m2024-07-17 13:49:18.238[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 4.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 3.1147e-05, 'ids_predicted': 6.0436e-06}[0m
 80%|████████  | 308/385 [04:48<01:06,  1.15it/s][32m2024-07-17 13:49:18.951[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 4.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 9.565e-09, 'ids_predicted': 3.3875e-09}[0m
 80%|████████  | 309/385 [04:48<01:02,  1.22it/s][32m2024-07-17 13:49:19.843[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 4.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 8.4854e-07, 'ids_predicted': 2.3019e-07}[0m
 81%|████████  | 310/385 [04:49<01:03,  1.19it/s][32m2024-07-17 13:49:20.487[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 5.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 6.1772e-06, 'ids_predicted': 1.8012e-06}[0m
 81%|████████  | 311/385 [04:50<00:57,  1.28it/s][32m2024-07-17 13:49:21.194[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 5.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 6.6628e-06, 'ids_predicted': 1.8077e-05}[0m
 81%|████████  | 312/385 [04:51<00:55,  1.32it/s][32m2024-07-17 13:49:21.864[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 5.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 6.462e-09, 'ids_predicted': 1.8686e-08}[0m
 81%|████████▏ | 313/385 [04:51<00:52,  1.36it/s][32m2024-07-17 13:49:22.764[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 5.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.3151e-05, 'ids_predicted': 8.4111e-06}[0m
 82%|████████▏ | 314/385 [04:52<00:55,  1.28it/s][32m2024-07-17 13:49:23.684[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 18.0, 'r_nw': 5.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.5114e-08, 'ids_predicted': 3.1721e-09}[0m
 82%|████████▏ | 315/385 [04:53<00:57,  1.21it/s][32m2024-07-17 13:49:24.340[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 2.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.7782e-05, 'ids_predicted': 1.6081e-05}[0m
 82%|████████▏ | 316/385 [04:54<00:53,  1.29it/s][32m2024-07-17 13:49:25.064[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 2.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.1881e-05, 'ids_predicted': 1.2722e-05}[0m
 82%|████████▏ | 317/385 [04:54<00:51,  1.32it/s][32m2024-07-17 13:49:26.864[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 2.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 3.8426e-06, 'ids_predicted': 2.2658e-06}[0m
 83%|████████▎ | 318/385 [04:56<01:11,  1.07s/it][32m2024-07-17 13:49:27.507[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 2.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 3.1664e-07, 'ids_predicted': 7.9097e-07}[0m
 83%|████████▎ | 319/385 [04:57<01:02,  1.06it/s][32m2024-07-17 13:49:28.209[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 2.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 5.0202e-06, 'ids_predicted': 1.7407e-05}[0m
 83%|████████▎ | 320/385 [04:58<00:56,  1.15it/s][32m2024-07-17 13:49:28.862[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 2.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 8.8049e-07, 'ids_predicted': 3.1513e-06}[0m
 83%|████████▎ | 321/385 [04:58<00:51,  1.24it/s][32m2024-07-17 13:49:29.568[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 2.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.5275e-05, 'ids_predicted': 1.324e-05}[0m
 84%|████████▎ | 322/385 [04:59<00:48,  1.29it/s][32m2024-07-17 13:49:30.212[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 2.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.3807e-09, 'ids_predicted': 9.1814e-09}[0m
 84%|████████▍ | 323/385 [05:00<00:45,  1.36it/s][32m2024-07-17 13:49:30.870[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 2.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 2.6896e-09, 'ids_predicted': 1.0071e-09}[0m
 84%|████████▍ | 324/385 [05:00<00:43,  1.40it/s][32m2024-07-17 13:49:31.661[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 2.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 7.1109e-09, 'ids_predicted': 4.5155e-08}[0m
 84%|████████▍ | 325/385 [05:01<00:44,  1.36it/s][32m2024-07-17 13:49:32.307[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 3.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 7.2943e-08, 'ids_predicted': 2.1239e-11}[0m
 85%|████████▍ | 326/385 [05:02<00:41,  1.41it/s][32m2024-07-17 13:49:33.206[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 3.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 2.5852e-05, 'ids_predicted': 2.1954e-05}[0m
 85%|████████▍ | 327/385 [05:03<00:44,  1.31it/s][32m2024-07-17 13:49:33.855[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 3.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.3976e-05, 'ids_predicted': 9.6492e-06}[0m
 85%|████████▌ | 328/385 [05:03<00:41,  1.37it/s][32m2024-07-17 13:49:34.612[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 3.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.3823e-06, 'ids_predicted': 6.2797e-07}[0m
 85%|████████▌ | 329/385 [05:04<00:41,  1.35it/s][32m2024-07-17 13:49:35.498[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 3.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 6.7072e-06, 'ids_predicted': 1.1745e-05}[0m
 86%|████████▌ | 330/385 [05:05<00:43,  1.28it/s][32m2024-07-17 13:49:36.206[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 3.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 6.8633e-06, 'ids_predicted': 1.3066e-05}[0m
 86%|████████▌ | 331/385 [05:06<00:41,  1.31it/s][32m2024-07-17 13:49:37.098[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 3.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 6.1437e-08, 'ids_predicted': 1.0533e-07}[0m
 86%|████████▌ | 332/385 [05:06<00:42,  1.25it/s][32m2024-07-17 13:49:37.998[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 3.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 8.5311e-06, 'ids_predicted': 3.6976e-06}[0m
 86%|████████▋ | 333/385 [05:07<00:43,  1.21it/s][32m2024-07-17 13:49:38.998[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 3.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 4.7453e-10, 'ids_predicted': 9.6266e-11}[0m
 87%|████████▋ | 334/385 [05:08<00:44,  1.14it/s][32m2024-07-17 13:49:39.696[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 3.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.3563e-05, 'ids_predicted': 8.4325e-06}[0m
 87%|████████▋ | 335/385 [05:09<00:41,  1.21it/s][32m2024-07-17 13:49:40.430[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 4.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 2.7042e-06, 'ids_predicted': 3.7663e-07}[0m
 87%|████████▋ | 336/385 [05:10<00:39,  1.25it/s][32m2024-07-17 13:49:41.202[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 4.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 7.6579e-08, 'ids_predicted': 1.3879e-09}[0m
 88%|████████▊ | 337/385 [05:11<00:37,  1.27it/s][32m2024-07-17 13:49:42.135[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 4.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 2.8554e-05, 'ids_predicted': 8.3167e-06}[0m
 88%|████████▊ | 338/385 [05:11<00:39,  1.20it/s][32m2024-07-17 13:49:42.816[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 4.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 3.8711e-07, 'ids_predicted': 4.5898e-07}[0m
 88%|████████▊ | 339/385 [05:12<00:36,  1.27it/s][32m2024-07-17 13:49:43.734[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 4.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 2.605e-05, 'ids_predicted': 1.3366e-05}[0m
 88%|████████▊ | 340/385 [05:13<00:37,  1.21it/s][32m2024-07-17 13:49:44.464[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 4.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.2221e-05, 'ids_predicted': 1.5225e-05}[0m
 89%|████████▊ | 341/385 [05:14<00:35,  1.25it/s][32m2024-07-17 13:49:45.126[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 4.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 4.3553e-06, 'ids_predicted': 5.1491e-06}[0m
 89%|████████▉ | 342/385 [05:14<00:32,  1.32it/s][32m2024-07-17 13:49:46.037[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 4.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 2.325e-05, 'ids_predicted': 2.7404e-05}[0m
 89%|████████▉ | 343/385 [05:15<00:33,  1.24it/s][32m2024-07-17 13:49:46.704[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 4.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.1908e-06, 'ids_predicted': 9.301e-07}[0m
 89%|████████▉ | 344/385 [05:16<00:31,  1.31it/s][32m2024-07-17 13:49:47.392[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 4.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 3.1761e-05, 'ids_predicted': 2.6921e-05}[0m
 90%|████████▉ | 345/385 [05:17<00:29,  1.35it/s][32m2024-07-17 13:49:48.159[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 5.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 3.5579e-06, 'ids_predicted': 4.9522e-07}[0m
 90%|████████▉ | 346/385 [05:17<00:29,  1.34it/s][32m2024-07-17 13:49:49.011[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 5.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.094e-06, 'ids_predicted': 8.802e-09}[0m
 90%|█████████ | 347/385 [05:18<00:29,  1.28it/s][32m2024-07-17 13:49:49.730[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 5.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 2.5319e-05, 'ids_predicted': 1.248e-05}[0m
 90%|█████████ | 348/385 [05:19<00:28,  1.31it/s][32m2024-07-17 13:49:50.418[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 5.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.8663e-05, 'ids_predicted': 2.1284e-05}[0m
 91%|█████████ | 349/385 [05:20<00:26,  1.35it/s][32m2024-07-17 13:49:51.113[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 19.0, 'r_nw': 5.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 2.0815e-06, 'ids_predicted': 2.1461e-07}[0m
 91%|█████████ | 350/385 [05:20<00:25,  1.38it/s][32m2024-07-17 13:49:51.774[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 2.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.0322e-05, 'ids_predicted': 9.5549e-06}[0m
 91%|█████████ | 351/385 [05:21<00:24,  1.42it/s][32m2024-07-17 13:49:52.444[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 2.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 9.1692e-07, 'ids_predicted': 7.1447e-09}[0m
 91%|█████████▏| 352/385 [05:22<00:22,  1.44it/s][32m2024-07-17 13:49:53.105[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 2.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 2.9114e-06, 'ids_predicted': 2.5728e-06}[0m
 92%|█████████▏| 353/385 [05:22<00:21,  1.46it/s][32m2024-07-17 13:49:53.751[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 2.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 3.8847e-06, 'ids_predicted': 1.7156e-07}[0m
 92%|█████████▏| 354/385 [05:23<00:20,  1.48it/s][32m2024-07-17 13:49:54.573[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 2.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.7846e-06, 'ids_predicted': 8.5315e-06}[0m
 92%|█████████▏| 355/385 [05:24<00:21,  1.39it/s][32m2024-07-17 13:49:55.254[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 2.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 1.095e-08, 'ids_predicted': 3.2284e-07}[0m
 92%|█████████▏| 356/385 [05:25<00:20,  1.41it/s][32m2024-07-17 13:49:55.961[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 2.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 5.274e-06, 'ids_predicted': 1.1041e-05}[0m
 93%|█████████▎| 357/385 [05:25<00:19,  1.41it/s][32m2024-07-17 13:49:56.608[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 2.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 6.1452e-06, 'ids_predicted': 9.0742e-06}[0m
 93%|█████████▎| 358/385 [05:26<00:18,  1.45it/s][32m2024-07-17 13:49:57.490[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 2.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 2.4384e-09, 'ids_predicted': 5.9594e-07}[0m
 93%|█████████▎| 359/385 [05:27<00:19,  1.34it/s][32m2024-07-17 13:49:58.381[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 2.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 2.3732e-06, 'ids_predicted': 1.2166e-06}[0m
 94%|█████████▎| 360/385 [05:28<00:19,  1.27it/s][32m2024-07-17 13:49:59.094[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 3.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 5.5682e-06, 'ids_predicted': 1.1321e-05}[0m
 94%|█████████▍| 361/385 [05:28<00:18,  1.30it/s][32m2024-07-17 13:50:01.753[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 3.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 7.9616e-10, 'ids_predicted': 3.026e-10}[0m
 94%|█████████▍| 362/385 [05:31<00:30,  1.33s/it][32m2024-07-17 13:50:02.475[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 3.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 5.4851e-07, 'ids_predicted': 2.6912e-06}[0m
 94%|█████████▍| 363/385 [05:32<00:25,  1.15s/it][32m2024-07-17 13:50:03.238[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 3.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 6.6514e-07, 'ids_predicted': 2.6591e-06}[0m
 95%|█████████▍| 364/385 [05:33<00:21,  1.03s/it][32m2024-07-17 13:50:03.904[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 3.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 7.3636e-10, 'ids_predicted': 1.5836e-10}[0m
 95%|█████████▍| 365/385 [05:33<00:18,  1.08it/s][32m2024-07-17 13:50:04.627[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 3.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 3.0984e-07, 'ids_predicted': 1.9737e-06}[0m
 95%|█████████▌| 366/385 [05:34<00:16,  1.16it/s][32m2024-07-17 13:50:05.313[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 3.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 7.1665e-07, 'ids_predicted': 3.5617e-07}[0m
 95%|█████████▌| 367/385 [05:35<00:14,  1.23it/s][32m2024-07-17 13:50:06.198[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 3.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 9.8314e-07, 'ids_predicted': 6.9636e-07}[0m
 96%|█████████▌| 368/385 [05:36<00:14,  1.20it/s][32m2024-07-17 13:50:06.859[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 3.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 4.2403e-06, 'ids_predicted': 6.4979e-06}[0m
 96%|█████████▌| 369/385 [05:36<00:12,  1.28it/s][32m2024-07-17 13:50:07.540[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 3.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.6426e-05, 'ids_predicted': 9.0202e-06}[0m
 96%|█████████▌| 370/385 [05:37<00:11,  1.33it/s][32m2024-07-17 13:50:08.310[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 4.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 8.6818e-09, 'ids_predicted': 3.4467e-10}[0m
 96%|█████████▋| 371/385 [05:38<00:10,  1.32it/s][32m2024-07-17 13:50:09.006[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 4.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.2402e-05, 'ids_predicted': 1.0597e-05}[0m
 97%|█████████▋| 372/385 [05:38<00:09,  1.35it/s][32m2024-07-17 13:50:09.822[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 4.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 4.7211e-06, 'ids_predicted': 1.106e-05}[0m
 97%|█████████▋| 373/385 [05:39<00:09,  1.31it/s][32m2024-07-17 13:50:10.535[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 4.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.0618e-05, 'ids_predicted': 1.1523e-05}[0m
 97%|█████████▋| 374/385 [05:40<00:08,  1.34it/s][32m2024-07-17 13:50:11.266[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 4.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.185e-05, 'ids_predicted': 2.5708e-05}[0m
 97%|█████████▋| 375/385 [05:41<00:07,  1.35it/s][32m2024-07-17 13:50:11.974[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 4.5, 't_ox_nm': 0.5, 'ids_grounding_truth': 3.4679e-06, 'ids_predicted': 1.0624e-06}[0m
 98%|█████████▊| 376/385 [05:41<00:06,  1.37it/s][32m2024-07-17 13:50:12.678[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 4.5, 't_ox_nm': 0.75, 'ids_grounding_truth': 1.6916e-05, 'ids_predicted': 1.3366e-05}[0m
 98%|█████████▊| 377/385 [05:42<00:05,  1.38it/s][32m2024-07-17 13:50:13.313[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 4.5, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.641e-05, 'ids_predicted': 1.3382e-05}[0m
 98%|█████████▊| 378/385 [05:43<00:04,  1.43it/s][32m2024-07-17 13:50:16.139[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 4.5, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.378e-05, 'ids_predicted': 1.3194e-05}[0m
 98%|█████████▊| 379/385 [05:45<00:08,  1.34s/it][32m2024-07-17 13:50:16.903[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 4.5, 't_ox_nm': 1.5, 'ids_grounding_truth': 4.2077e-06, 'ids_predicted': 2.0124e-06}[0m
 99%|█████████▊| 380/385 [05:46<00:05,  1.16s/it][32m2024-07-17 13:50:17.813[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 5.0, 't_ox_nm': 0.5, 'ids_grounding_truth': 7.2941e-07, 'ids_predicted': 3.3443e-08}[0m
 99%|█████████▉| 381/385 [05:47<00:04,  1.09s/it][32m2024-07-17 13:50:18.689[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 5.0, 't_ox_nm': 0.75, 'ids_grounding_truth': 2.4342e-07, 'ids_predicted': 2.1763e-07}[0m
 99%|█████████▉| 382/385 [05:48<00:03,  1.02s/it][32m2024-07-17 13:50:19.354[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 5.0, 't_ox_nm': 1.0, 'ids_grounding_truth': 1.7479e-06, 'ids_predicted': 6.7386e-07}[0m
 99%|█████████▉| 383/385 [05:49<00:01,  1.09it/s][32m2024-07-17 13:50:20.250[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 5.0, 't_ox_nm': 1.25, 'ids_grounding_truth': 1.5309e-05, 'ids_predicted': 1.5436e-05}[0m
100%|█████████▉| 384/385 [05:50<00:00,  1.10it/s][32m2024-07-17 13:50:21.092[0m | [1mINFO    [0m | [36m__main__[0m:[36m<module>[0m:[36m24[0m - [1mResult: {'lg_nw': 20.0, 'r_nw': 5.0, 't_ox_nm': 1.5, 'ids_grounding_truth': 1.0939e-05, 'ids_predicted': 3.3029e-06}[0m
100%|██████████| 385/385 [05:50<00:00,  1.10it/s]
"""

In [38]:
res_string_list = res_string.split("\n")
len(res_string_list)

387

In [39]:
def extract_data(input_string):
    # 定义正则表达式模式
    pattern = r"'(\w+)':\s*([\d.e-]+)"
    
    # 使用正则表达式查找所有匹配项
    matches = re.findall(pattern, input_string)
    
    # 创建一个字典来存储提取的数据
    extracted_data = {}
    
    # 遍历匹配项并将它们添加到字典中
    for key, value in matches:
        # 将字符串值转换为适当的数值类型
        if 'e' in value:
            extracted_data[key] = float(value)
        elif '.' in value:
            extracted_data[key] = float(value)
        else:
            extracted_data[key] = int(value)
    
    return extracted_data
res = [extract_data(item) for item in res_string_list if item]

In [41]:
res = [r for r in res if r]

In [44]:
save_path = RESOURCES_DIR_PATH / "evaluation_results.jsonl"
with open(save_path, "w") as f:
    for r in res:
        f.write(json.dumps(r) + "\n")